> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 4.2 - APIs


## Exercícios 🔭🌌🪐

Vamos acessar as APIs da NASA para ver algumas imagens interessantes capturadas universo afora!

#### Q1.
Crie uma chave no site oficial:
* https://api.nasa.gov

Vamos armazenar a chave de forma segura! <br>
Salve a sua chave em um arquivo `key.json` na forma:
`API_KEY=SUA_CHAVE`

Adicione o nome do arquivo `key.json` ao `.gitignore` do repositório que você fará upload da atividade.
Para isso basta abrir o arquivo `.gitignore` na pasta raíz do repositório (ou criar um caso ele não exista). Dentro do arquivo, apenas adiciona o nome do arquivo que deseja ignorar.

#### Q2. 🛰 Astronomy Picture of the Day (APOD) 🌌
> Antes de fazer os exercícios, devo te lembrar que existem limites de acesso às APIs, descritas na página principal, portanto pega leve na tentativa e erro na hora de testar seu código.

<img width=500 src=https://apod.nasa.gov/apod/image/2407/M24-HaLRGB-RC51_1024.jpg>

A primeira API que acessaremos é a mais popular de todas: astronomy picture of the day (foto astronômica do dia).

Faça uma requisição GET para a URL da API que retorna a imagem do dia! Essa é fácil já que são os valores padrão da rota principal:
* URL base: `'https://api.nasa.gov/planetary/apod'`
* Endpoint: não precisa preencher, acessaremos a raíz da API.
* Query params: preencha `api_key` com a sua chave de autenticação. Se animar mexer em outros parâmetros veja [a documentação](https://api.nasa.gov).

Ao receber a resposta (um json), você deve:
* Imprimir os campos `copyright` e `explanation`
* Com as biblioteca scikit-images e matplotlib, apresente a imagem a partir do campo `url` ou `hdurl`, e preencha o título do plot com o campo `title` do json. Uma dica de código a seguir.
```python
from skimage import io
img = io.imread(url)
## plot a matriz img com matplotlib (imshow)
```   

In [ ]:
import requests
import json
from skimage import io
import matplotlib.pyplot as plt

# Carregar a chave de autenticação do arquivo key.json
def load_api_key():
    with open("key.json", "r") as file:
        data = json.load(file)
        return data["API_KEY"]

# Fazer a requisição GET para a API
def fetch_apod(api_key):
    url = "https://api.nasa.gov/planetary/apod"
    params = {"api_key": api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Erro ao acessar a API: {response.status_code}, {response.text}")

# Mostrar a imagem com matplotlib
def display_image(url, title):
    img = io.imread(url)  # Carregar a imagem
    plt.imshow(img)
    plt.axis("off")
    plt.title(title, fontsize=16)
    plt.show()

# Fluxo principal
try:
    api_key = load_api_key()  # Carregar chave da API
    apod_data = fetch_apod(api_key)  # Buscar dados da API
    
    # Imprimir os campos copyright e explanation
    print("Copyright:", apod_data.get("copyright", "N/A"))
    print("Explanation:", apod_data["explanation"])
    
    # Exibir a imagem
    image_url = apod_data.get("hdurl") or apod_data["url"]
    display_image(image_url, apod_data["title"])

except Exception as e:
    print(f"Erro: {e}")


#### Q3. Limites
A partir da resposta da query anterios, imprima o header da resposta e consulte os atributos:
* X-RateLimit-Limit: o limite total de requisições da sua chave de API
* X-RateLimit-Remaining: o limite restante de requisições da sua chave de API

In [ ]:
import requests
import json

# Função para carregar a chave da API
def load_api_key():
    with open("key.json", "r") as file:
        data = json.load(file)
    return data["API_KEY"]

# Função para realizar a requisição GET e imprimir os cabeçalhos
def fetch_mars_photos_with_limits(api_key, rover_name, sol, page):
    url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/{rover_name}/photos'
    params = {
        'api_key': api_key,
        'sol': sol,
        'page': page
    }
    
    response = requests.get(url, params=params)
    
    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Imprimir os cabeçalhos de resposta
        print("Cabeçalhos da resposta:")
        print(response.headers)
        
        # Imprimir os limites de requisição
        rate_limit = response.headers.get('X-RateLimit-Limit')
        rate_remaining = response.headers.get('X-RateLimit-Remaining')
        
        if rate_limit and rate_remaining:
            print(f"\nLimite total de requisições: {rate_limit}")
            print(f"Limite restante de requisições: {rate_remaining}")
        
        # Retornar os dados da API
        return response.json()
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Fluxo principal para testar a função
def main():
    api_key = load_api_key()  # Carregar chave da API
    rover_name = 'curiosity'  # Nome do rover
    sol = 4102  # Sol desejado
    page = 1  # Página inicial
    
    # Chamar a função e obter os dados
    data = fetch_mars_photos_with_limits(api_key, rover_name, sol, page)
    
    if data:
        # Aqui você pode continuar processando os dados da resposta, se necessário
        print(f"\nFotos encontradas: {len(data['photos'])}")
    else:
        print("Nenhuma foto encontrada.")

if __name__ == '__main__':
    main()


### Q4. Mars Rover Photos 🚀🚙 📷

<img width=500 src=https://www.nasa.gov/wp-content/uploads/2019/10/pia23378-16.jpg>

Essa API retorna dados (incluindo imagens capturadas) sobre os veículos que hoje habitam o planeta Marte. São os rovers `opportunity`, `spirit` e o mais famoso, o `curiosity` (da foto acima).

Antes de requisitar imagens, vamos ver o relatório de dados coletados por um deles, o `curiosity`. Isso vai nos ajudar a montar a query de imagens coletadas.

Faça uma requisição GET para a seguinte URL:
* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* endpoint: `'/manifests/{nome_do_rover}'`
* query parameters: preencha `api_key` com a sua chave de autenticação.

Extraia o json da resposta retornada. O campo principal é o `'photo_manifest'`, do qual queremos acessar os seguintes valores:
* `max_sol`: Máximo "dia marciano" de coleta de fotos. O dia marciano tem 24 horas, 39 minutos e 35 segundos.
* `max_date`: Última data terrestre de coleta de fotos, na forma `'aaaa-mm-dd'`.

Imprima esses dois atributos da resposta e os use no próximo exercício para coletar as fotos mais recentes tiradas. 

In [ ]:
import requests

# Função para carregar a chave da API do arquivo JSON
def load_api_key():
    with open("key.json", "r") as file:
        data = json.load(file)
    return data["API_KEY"]

# Função para fazer a requisição GET para a API
def fetch_apod(api_key):
    url = "https://api.nasa.gov/planetary/apod"
    params = {"api_key": api_key}
    response = requests.get(url, params=params)

    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Imprimir os cabeçalhos
        print("Cabeçalhos da resposta:")
        print(response.headers)

        # Acessar os limites de requisição
        rate_limit = response.headers.get('X-RateLimit-Limit')
        rate_remaining = response.headers.get('X-RateLimit-Remaining')
        print(f"Limite total de requisições: {rate_limit}")
        print(f"Limite restante de requisições: {rate_remaining}")

        return response.json()
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Fluxo principal
try:
    api_key = load_api_key()  # Carregar chave da API
    apod_data = fetch_apod(api_key)  # Buscar dados da API

    # Exibir os dados da API (exemplo)
    if apod_data:
        print("Data da APOD:", apod_data.get("date"))
        print("Título:", apod_data.get("title"))
        print("Explicação:", apod_data.get("explanation"))
        # Exibir a imagem (se disponível)
        image_url = apod_data.get("hdurl") or apod_data.get("url")
        print("URL da imagem:", image_url)

except Exception as e:
    print(f"Erro: {e}")


#### Q5.

Faça uma requisição GET para a URL da API que retorna links para as imagens coletadas pelos rovers.

* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* Endpoint: `/rovers/{nome_do_rover}/photos`
* Query params sugeridos: 
    * `api_key`: sua chave de autenticação.
    * `sol`: dia marciano que deseja coletar (de 0 a `max_sol` coletado anteriormente)
    * `page`: você pode paginar entre as respostas! São retornados 25 resultados por página.

A resposta esperada estará no formato a seguir, uma lista no campo `'photos'` onde cada item é um dicionário com os dados da foto tirada. Dentre os dados há o campo `camera` indicando qual das câmeras do rover tirou a foto. As fotos mais interessantes (na minha opinião, claro) são das câmeras de navegação (`"name": "NAVCAM"`) e as de prevenção de colisão (frente: `"name": "FHAZ"` e trás `"name": "RHAZ"`) onde dá pra ver partes do robô!

**Seu trabalho é**:
* Paginar a requisição acima até que a resposta seja `None`
* Escolher uma ou mais câmeras (ex: `NAVCAM`, `FHAZ`, `RHAZ`), e em um laço de repetição plotar todas as imagens retornadas daquela câmera. Use novamente as bibliotecas scikit-image e matplotlib. 
  * O título da imagem deve ter a página da requisição, nome da câmera e id da imagem.

```json
{
  "photos": [
    {
      "id": 1228212,
      "sol": 4102,
      "camera": {
        "id": 20,
        "name": "FHAZ",
        "rover_id": 5,
        "full_name": "Front Hazard Avoidance Camera"
      },
      "img_src": "https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/04102/opgs/edr/fcam/FLB_761645828EDR_F1060660FHAZ00302M_.JPG",
      "earth_date": "2024-02-19",
      "rover": {
        "id": 5,
        "name": "Curiosity",
        ...
      }
    }
    {
      "id": 1228213,
      "sol": 4102, 
      ...
    }
```



import requests
import matplotlib.pyplot as plt
from skimage.io import imread

# Função para carregar a chave da API
def load_api_key():
    with open("key.json", "r") as file:
        data = json.load(file)
    return data["API_KEY"]

# Função para buscar fotos dos rovers
def fetch_mars_photos(api_key, rover_name, sol, page):
    url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/{rover_name}/photos'
    params = {
        'api_key': api_key,
        'sol': sol,
        'page': page
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Função para exibir as imagens filtradas por câmera
def display_images(photos, camera_name, page):
    # Filtrar fotos pela câmera desejada
    filtered_photos = [photo for photo in photos if photo['camera']['name'] == camera_name]
    
    for photo in filtered_photos:
        img_url = photo['img_src']
        img = imread(img_url)
        
        # Título com o número da página, nome da câmera e id da imagem
        title = f"Page {page} - {camera_name} - ID {photo['id']}"
        
        # Plotar a imagem
        plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')
        plt.show()

# Fluxo principal para paginar e exibir as imagens
def main():
    api_key = load_api_key()  # Carregar chave da API
    rover_name = 'curiosity'  # Nome do rover (ex: curiosity)
    sol = 4102  # Sol desejado (dia marciano)
    page = 1  # Página inicial
    
    while True:
        # Buscar fotos do rover na página atual
        data = fetch_mars_photos(api_key, rover_name, sol, page)
        
        if data and 'photos' in data and len(data['photos']) > 0:
            photos = data['photos']
            
            # Exibir imagens das câmeras NAVCAM, FHAZ, e RHAZ
            for camera_name in ['NAVCAM', 'FHAZ', 'RHAZ']:
                display_images(photos, camera_name, page)
            
            # Incrementar a página para próxima requisição
            page += 1
        else:
            print("Nenhuma foto encontrada ou atingido o final das páginas.")
            break

if __name__ == '__main__':
    main()
